In [1]:
import os
import pathlib
import sys

import numpy as np
import pandas as pd
from IPython.display import HTML, display, display_html

mod_path = os.path.abspath(os.path.join("../../src/alyslib"))
if mod_path not in sys.path:
    sys.path.append(mod_path)

import alyslib

In [2]:
l = alyslib.import_data("./data", "net.gen")
d0, d1 = alyslib.build_dfs(l)
dfmerge = d0 + d1
alyslib.clean_network_noise(dfmerge)
alyslib.sort_by_key(dfmerge, "Timestamp")
alyslib.cmp_elapsed(dfmerge)

dataframe 139905225226272, removed 3 items, [307, 304, 306]
dataframe 139905225226272, removed 1 items, [305]
dataframe 139905225227376, removed 3 items, [22678, 22677, 22679]
dataframe 139905225227376, removed 1 items, [22680]
dataframe 139905222317376, removed 3 items, [22993, 22990, 22992]
dataframe 139905222317376, removed 1 items, [22991]
dataframe 139905222319536, removed 3 items, [12584, 12583, 12585]
dataframe 139905222319536, removed 1 items, [12586]
dataframe 139905222318480, removed 143 items, [12019, 12018, 12020, 537, 538]
dataframe 139905222318480, removed 122 items, [12021, 539, 541, 543, 546]
dataframe 139905222314496, removed 143 items, [24477, 24474, 24476, 2834, 2835]
dataframe 139905222314496, removed 121 items, [24475, 2836, 2838, 2840, 2843]
dataframe 139905222327216, removed 3 items, [24427, 24425, 24426]
dataframe 139905222327216, removed 1 items, [24428]
dataframe 139905222312384, removed 3 items, [12678, 12677, 12679]
dataframe 139905222312384, removed 1 items

In [4]:
def get_means(df):
    l = []
    for d in df:
        l.append(d.TimeDelta.mean())
    return l


def conf_interval(data, z_score=1.96):
    mean = np.mean(data)
    std = np.std(data)
    size = len(data)
    err = z_score * (std / np.sqrt(size))
    return (mean - err, mean + err)


d0m = get_means(d0)
display(pd.DataFrame(d0m, columns=["means"]))
display(
    pd.DataFrame(
        [[np.mean(d0m), np.std(d0m), conf_interval(d0m)]],
        columns=["mean of means", "std of means", "95% conf interval"],
    )
)

d1m = get_means(d1)
display(pd.DataFrame(d1m, columns=["means"]))
display(
    pd.DataFrame(
        [[np.mean(d1m), np.std(d1m), conf_interval(d1m)]],
        columns=["mean of means", "std of means", "95% conf interval"],
    )
)

,means
0,0.004381
1,0.004981
2,0.004599
3,0.004641
4,0.005199
5,0.006616
6,0.005071
7,0.005243
8,0.004792
9,0.004910


,mean of means,std of means,95% conf interval
0,0.005043,0.000585,"(0.0046807860864946736, 0.005405812325444093)"


,means
0,0.019003
1,0.018425
2,0.018879
3,0.018740
4,0.018031
5,0.018990
6,0.018123
7,0.018329
8,0.017980
9,0.018411


,mean of means,std of means,95% conf interval
0,0.018491,0.00037,"(0.0182618713257612, 0.01872026641485766)"
